# CS 421 PROJECT
---

# Data Loading

In [17]:
import numpy as np
import pandas as pd

In [ ]:
data  = np.load("data/regression_labelled.npz")
X     = data["X"]
y     = data["yy"]
y_cat = data["yy_cat"]

# Load dataframes
X     = pd.DataFrame(X, columns=["user", "item", "rating"])
y     = pd.DataFrame(y, columns=["user", "label"])
y_cat = pd.DataFrame(y_cat, columns=["user", "label", "anomtype"])

# Parse to correct types
y     = y.astype({"user": int, "label": float})
y_cat = y_cat.astype({"user": int, "label": float, "anomtype": int})

In [ ]:
XX    = np.load("data/regression_unlabelled.npz")['X']
XX    = pd.DataFrame(XX, columns=["user", "item", "rating"])

# Feature Engineering  
To estimate the noise level $p$ for each user, we extract user-level statistical features that capture consistency, randomness, and alignment with item averages in each user’s rating behavior.

---

### **Feature Intuition for Noise Level Prediction**

- **Mean Rating** – Represents a user’s overall rating tendency.  
  High-noise users often have averages close to the dataset’s midpoint (≈ 3) due to random rating behavior.  

- **Standard Deviation of Ratings** – Captures the variability in a user’s ratings.  
  A higher standard deviation generally indicates greater randomness and, therefore, higher noise.  

- **Rating Range** – Reflects how widely a user utilizes the rating scale.  
  Noisy users tend to rate across the full 1–5 range.  

- **Number of Ratings** – Indicates how many items a user has rated.  
  The noise generation process may influence this count, providing additional clues about the user’s noise level.  

- **Median Rating** – Provides a robust measure of central tendency.  
  Unlike the mean, it is less affected by extreme ratings.

- **Median Absolute Deviation of Ratings** – Measures the spread of a user's ratings around their central tendency.
  Low noise users should have more consistent biases to lower (1/2) and upper (4/5) ends while high noise users' ratings are more dispersed.

- **Mean Item Popularity** – Represents the overall popularity of items a user rates.
  Helps differentiate between users who seek out niche content compared to mainstream content

- **Skewness of Ratings** – Measures asymmetry in the rating distribution.  
  Helps identify users whose ratings are biased toward low or high values.

- **Kurtosis of Ratings** – Measures the peakedness of the rating distribution.  
  High kurtosis may indicate that ratings cluster around certain values.

- **Kurtosis Ratio of Ratings** – Measures the ratio of peakedness of the rating distribution to the variability in a user's ratings.  
  Helps with noise that avoid the midpoint (3) in ratings.

- **Rating Entropy** – Captures the randomness or unpredictability in a user’s ratings.  
  Higher entropy generally indicates higher noise.

- **Item Average Entropy** – Captures the randomness or unpredictability in what items a user rates.  
  Higher entropy generally indicates higher noise.

- **Proportion of Extreme Ratings (1★ or 5★)** – Fraction of very low or very high ratings.  
  High-noise users often have more evenly spread ratings, while consistent users may avoid extremes.

- **Proportion of Extreme Disagreement** – Fraction of very low or very high ratings given to items that are generally not considered as such.  
  High-noise users would not care, while consistent users would join the majority.

- **Proportion of Unique Ratings (1★ or 5★)** – Fraction of the ratings a user has given out of the total (5).  
  High-noise users would use more ratings, while consistent users may have a bias towards one end.

- **Fraction of Consecutive Repeated Ratings** – Measures how often a user gives the same rating consecutively.  
  Can indicate structured versus random behavior.

- **Mean Deviation from Item Average** – Average difference between a user’s rating and the corresponding item’s mean rating.  
  Noisy users tend to have ratings closer to 0 deviation, while structured users may systematically deviate.

- **Standard Deviation of Deviations** – Measures variability in how a user deviates from item averages.  
  Higher values indicate less predictable alignment with item norms.

- **Average Absolute Deviation from Item Mean** – Average of the absolute deviations, providing a robust measure of alignment with typical item ratings.  
  Captures overall disagreement with item norms, which can help identify noisy behavior.

- **Median Absolute Deviation from Item Mean** – Median of the absolute deviations, providing a robust measure of alignment with typical item ratings.  
  Captures overall consistency of disagreement with item norms, which can help identify noisy behavior.


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, entropy

# Function to compute per-user features including deviation from item mean
def extract_user_features(df):
    # Compute global item average
    item_mean = df.groupby("item")["rating"].mean().rename("global_item_mean")
    item_popularity = df.groupby("item").size().rename("popularity")

    df = df.merge(item_mean, on="item", how="left")
    df = df.merge(item_popularity, on="item", how="left")
    
    # Compute deviation from item mean for each rating
    df["dev_from_item_mean"] = df["rating"] - df["global_item_mean"]
    df["item_mean_rounded"] = df["global_item_mean"].round().astype(int)
    df["match_item_mean"] = (df["rating"] == df["item_mean_rounded"]).astype(int)
    df["log_popularity"] = np.log(df["popularity"] + 1)

    df = df.drop(columns=["global_item_mean", "popularity"])
    
    features_list = []

    for user_id, group in df.groupby("user"):
        ratings = group["rating"].values
        devs = group["dev_from_item_mean"].values
        kurt_val = kurtosis(ratings)
        
        # Basic statistics
        mean_rating = ratings.mean()
        std_rating = ratings.std()
        min_rating = ratings.min()
        max_rating = ratings.max()
        rating_range = max_rating - min_rating
        num_ratings = len(ratings)
        median_rating = np.median(ratings)

        # 1. Calculate median of absolute deviations
        user_median = np.median(ratings)
        abs_dev_from_median = np.abs(ratings - user_median)
        mad_rating = np.median(abs_dev_from_median)

        if std_rating == 0:
            # If spread is 0, the distribution is perfectly peaked.
            kurtosis_ratio = 100.0 
        else:
            kurtosis_ratio = kurt_val / std_rating

        mean_item_popularity = group["log_popularity"].mean()
        
        # Advanced statistics
        skewness = skew(ratings)
        
        # Entropy of rating distribution
        counts = np.bincount(ratings, minlength=6)[1:]  # ratings 1-5
        prob = counts / counts.sum()
        rating_entropy = entropy(prob)

        # Entropy of item rating distribution
        item_avg_ratings = group["item_mean_rounded"].values
        item_avg_counts = np.bincount(item_avg_ratings, minlength=6)[1:] 
        item_avg_prob = item_avg_counts / item_avg_counts.sum()
        item_avg_entropy = entropy(item_avg_prob)
        
        # Proportion of extreme ratings
        prop_1 = np.sum(ratings == 1) / num_ratings
        #prop_2 = np.sum(ratings == 2) / num_ratings
        prop_3 = np.sum(ratings == 3) / num_ratings
        prop_4 = np.sum(ratings == 4) / num_ratings
        prop_5 = np.sum(ratings == 5) / num_ratings

        # Proportion of unique ratings used
        prop_unique_ratings = len(np.unique(ratings)) / num_ratings

        # Proportion of disagreement, rated extremely when item is not
        is_extreme_rating = (group["rating"] == 1) | (group["rating"] == 5)
        is_not_extreme_item = (group["item_mean_rounded"] > 1) & (group["item_mean_rounded"] < 5)
        disagreement_count = np.sum(is_extreme_rating & is_not_extreme_item)
        prop_extreme_disagree = disagreement_count / num_ratings

        # Proportion of ratings that match rounded item mean
        prop_match_item_mean = group["match_item_mean"].mean()
        
        # Fraction of repeated consecutive ratings
        repeats = np.sum(ratings[1:] == ratings[:-1]) / (num_ratings - 1) if num_ratings > 1 else 0
        
        abs_devs = np.abs(devs)
        mean_dev = devs.mean()
        std_dev = devs.std()
        abs_mean_dev = np.abs(devs).mean()
        med_abs_dev_item = np.median(abs_devs)
        
        features_list.append({
            "user": user_id,
            "mean_rating": mean_rating,
            "std_rating": std_rating,
            #"min_rating": min_rating,
            #"max_rating": max_rating,
            #"rating_range": rating_range,
            "num_ratings": num_ratings,
            "median_rating": median_rating,
            "mad_rating": mad_rating,
            "mean_item_popularity": mean_item_popularity,
            "skewness": skewness,
            "kurtosis": kurt_val,
            #"kurtosis_ratio": kurtosis_ratio,
            "rating_entropy": rating_entropy,
            "item_avg_entropy": item_avg_entropy,
            "prop_1": prop_1,
            "prop_3": prop_3,
            "prop_5": prop_5,
            "prop_extreme_disagree": prop_extreme_disagree,
            #"prop_unique_ratings": prop_unique_ratings,
            "prop_match_item_mean": prop_match_item_mean,
            #"repeat_fraction": repeats,
            "mean_dev": mean_dev,
            "std_dev": std_dev,
            "abs_mean_dev": abs_mean_dev,
            "med_abs_dev_item": med_abs_dev_item,
           "cv_rating": std_rating / mean_rating if mean_rating != 0 else 0,

        })
    
    features_df = pd.DataFrame(features_list)
    return features_df

# Apply to train and test datasets
X_feat = extract_user_features(X)
XX_feat = extract_user_features(XX)

# Merge with labels for training
train_df = pd.merge(X_feat, y, on="user")
train_df.head()


,user,mean_rating,std_rating,num_ratings,median_rating,mad_rating,mean_item_popularity,skewness,kurtosis,rating_entropy,...,prop_3,prop_5,prop_extreme_disagree,prop_match_item_mean,mean_dev,std_dev,abs_mean_dev,med_abs_dev_item,cv_rating,label
0,2700,2.304615,1.246340,325,2.0,1.0,5.780305,0.010275,-0.807643,1.452700,...,0.240000,0.021538,0.236923,0.240000,-0.900648,1.181644,1.231052,1.067416,0.540802,0.496169
1,2701,3.626898,0.522439,461,4.0,0.0,5.656944,-0.566175,-0.658135,0.754231,...,0.360087,0.008677,0.008677,0.427332,0.345794,0.613991,0.577960,0.601093,0.144046,0.348425
2,2702,3.339286,1.165375,224,3.5,0.5,5.838453,-0.886810,0.813582,1.374229,...,0.321429,0.129464,0.169643,0.290179,0.236531,1.127293,0.919958,0.771128,0.348989,0.111702
3,2703,3.619658,1.119275,234,4.0,0.0,5.781465,-1.022916,0.335700,1.320352,...,0.132479,0.170940,0.256410,0.188034,0.358184,1.132465,1.011778,0.848164,0.309221,0.343628
4,2704,2.572414,1.137116,435,3.0,1.0,5.742189,-0.103944,-0.865041,1.440866,...,0.291954,0.018391,0.206897,0.291954,-0.580245,1.140364,1.045617,0.943522,0.442042,0.625995


# Task 1: Train Regression Model

##### We first train on **Linear Regression** and **Polynomial Regression** and compare results.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# --- Prepare features and target ---
feature_cols = [c for c in X_feat.columns if c != "user"]
X_features = X_feat[feature_cols]
y_labels = y["label"]

# --- Train-validation split ---
X_train, X_val, y_train, y_val = train_test_split(
    X_features, y_labels, test_size=0.2, random_state=42
)

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# --- Linear Regression ---
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = np.clip(lin_reg.predict(X_val_scaled), 0, 1)  # clip to [0,1]
mae_lin = mean_absolute_error(y_val, y_pred_lin)

# --- Polynomial Regression (Degree 2) ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_val_poly = poly.transform(X_val_scaled)

poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)
y_pred_poly = np.clip(poly_reg.predict(X_val_poly), 0, 1)  # clip to [0,1]
mae_poly = mean_absolute_error(y_val, y_pred_poly)

# --- Compare Results ---
results_base = pd.DataFrame({
    "Model": ["Linear Regression", "Polynomial Regression (deg=2)"],
    "MAE": [mae_lin, mae_poly]
}).sort_values(by="MAE", ascending=True, ignore_index=True)

print("Baseline Model Comparison:")
print(results_base)


Baseline Model Comparison:
                           Model       MAE
0  Polynomial Regression (deg=2)  0.071884
1              Linear Regression  0.108613


#### Choosing a Regularizer
The polynomial regression (deg=2) model showed improved performance over the simple linear regression baseline, indicating that nonlinear relationships exist in the data.  
To further enhance generalization and reduce potential overfitting from the polynomial terms, we now apply **regularized polynomial models** — Ridge, Lasso, and ElasticNet — which introduce penalty terms to balance model complexity and predictive accuracy.


In [11]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV

# --- Polynomial Ridge Regression ---
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_poly, y_train)
y_pred_ridge = np.clip(ridge.predict(X_val_poly), 0, 1)  # clip to [0,1]
mae_ridge = mean_absolute_error(y_val, y_pred_ridge)

# --- Polynomial Lasso Regression ---
lasso = Lasso(alpha=0.001, max_iter=10000)
lasso.fit(X_train_poly, y_train)
y_pred_lasso = np.clip(lasso.predict(X_val_poly), 0, 1)
mae_lasso = mean_absolute_error(y_val, y_pred_lasso)

# --- Polynomial ElasticNet Regression ---
elastic = ElasticNet(alpha=0.001, l1_ratio=0.255, max_iter=10000)
elastic.fit(X_train_poly, y_train)
y_pred_elastic = np.clip(elastic.predict(X_val_poly), 0, 1)
mae_elastic = mean_absolute_error(y_val, y_pred_elastic)

# --- Compare Results ---
results_reg = pd.DataFrame({
    "Model": [
        "Polynomial Ridge",
        "Polynomial Lasso",
        "Polynomial ElasticNet"
    ],
    "MAE": [mae_ridge, mae_lasso, mae_elastic]
}).sort_values(by="MAE", ascending=True, ignore_index=True)

print("Regularized Polynomial Model Comparison:")
print(results_reg)


Regularized Polynomial Model Comparison:
                   Model       MAE
0  Polynomial ElasticNet  0.064539
1       Polynomial Ridge  0.070188
2       Polynomial Lasso  0.070390


### Conclusion

After comparing several regression approaches for predicting user noise level \(p\):

- **Linear Regression** provided a simple baseline but struggled with nonlinear interactions (MAE ≈ 0.1145).  
- **Polynomial Regression (degree 2)** captured non-linearities and improved performance (MAE ≈ 0.0948).  
- **Regularized Polynomial Models** (Ridge, Lasso, ElasticNet) were tested to control overfitting. Among these, **Polynomial ElasticNet Regression** achieved the **best performance** with **MAE ≈ 0.0782**, demonstrating that combining polynomial features with L1 and L2 regularization effectively balances complexity and generalization.

Thus, **Polynomial ElasticNet (degree 2)** is the recommended model for predicting user noise levels in this dataset.


# Task 2: Semi-Supervised Anomaly Type Prediction

We predict the anomaly type $X$  using a semi-supervised approach:

1. **Feature Preparation** – Use the same user-level statistical features as for regression.
2. **Standardization** – Scale features to zero mean and unit variance for effective clustering.
3. **Unsupervised Clustering** – Apply K-Means with 3 clusters to group users based on similarity in their feature profiles (distance in feature space).
4. **Cluster-to-Class Mapping** – Use the small labeled set (`y_cat`) to assign cluster labels to actual anomaly types.
5. **Prediction for All Users** – Assign predicted classes to all users, handling clusters without labeled users using nearest cluster centers.


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.mixture import GaussianMixture
from sklearn.semi_supervised import LabelSpreading

# --- Prepare features ---
feature_cols = [c for c in X_feat.columns if c != "user"]
X_train_features = X_feat[feature_cols]
X_train_users = X_feat["user"]

# Optional: polynomial features (degree 2)
poly_degree = 2
poly = PolynomialFeatures(degree=poly_degree, include_bias=False)
X_train_poly = poly.fit_transform(X_train_features)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)

# --- Step 1: Fit Gaussian Mixture Model ---
n_clusters = 3
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
cluster_labels = gmm.fit_predict(X_train_scaled)

# Map clusters to anomaly types using labeled users
cluster_df = pd.DataFrame({
    "user": X_train_users,
    "cluster": cluster_labels
})

cluster_to_class = {}
for cluster in np.unique(cluster_labels):
    users_in_cluster = cluster_df[cluster_df["cluster"] == cluster]["user"]
    labeled_in_cluster = y_cat[y_cat["user"].isin(users_in_cluster)]
    
    if not labeled_in_cluster.empty:
        counts = labeled_in_cluster["anomtype"].value_counts()
        max_count = counts.max()
        candidates = counts[counts == max_count].index.tolist()
        cluster_to_class[cluster] = min(candidates)  # tie-breaker
    else:
        cluster_to_class[cluster] = -1

cluster_df["predicted_anomtype"] = cluster_df["cluster"].map(cluster_to_class)

# --- Step 2: Label Spreading for semi-supervised refinement ---
# Prepare labels: -1 for unlabeled users
labels = np.full(X_train_scaled.shape[0], -1)
labeled_users_idx = X_feat["user"].isin(y_cat["user"])
labels[labeled_users_idx] = y_cat.set_index("user").loc[X_feat["user"][labeled_users_idx], "anomtype"].values

# Fit Label Spreading
label_model = LabelSpreading(kernel='knn', alpha=0.8, max_iter=1000)
label_model.fit(X_train_scaled, labels)
predicted_anomtype = label_model.transduction_

# --- Step 3: Predict anomaly type for test users ---
X_test_features = XX_feat[feature_cols]
X_test_poly = poly.transform(X_test_features)
X_test_scaled = scaler.transform(X_test_poly)

# Predict cluster probabilities for test users using GMM
test_probs = gmm.predict_proba(X_test_scaled)
# Assign cluster based on max probability
test_cluster_labels = np.argmax(test_probs, axis=1)
# Map to anomaly type using cluster_to_class
test_pred_class = [cluster_to_class.get(c, -1) for c in test_cluster_labels]

# Use Label Spreading to refine test predictions
test_pred_refined = label_model.predict(X_test_scaled)

# Combine regression and refined classification for submission
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_scaled, y["label"])
predicted_p_test = np.clip(ridge.predict(X_test_scaled), 0, 1)

final_predictions = pd.DataFrame({
    "user": XX_feat["user"],
    "predicted_p": predicted_p_test,
    "predicted_anomtype": test_pred_refined  # refined by label spreading
})

# Save submission
submission_file = "week1234_submission.csv"
final_predictions.to_csv(submission_file, index=False)
print(f"Submission saved to {submission_file}")

Submission saved to week1234_submission.csv


# Qualitative analysis of each anomaly class type

In [13]:
import pandas as pd
from scipy.stats import skew, kurtosis

# Load CSV
df = pd.read_csv("week1234_submission.csv")

# Function to compute extra stats
def prop_low(series, threshold=0.2):
    return (series <= threshold).sum() / len(series)

def prop_high(series, threshold=0.8):
    return (series >= threshold).sum() / len(series)

# Aggregate per anomaly type
summary_table = df.groupby("predicted_anomtype")["predicted_p"].agg(
    count='count',
    min='min',
    max='max',
    median='median',
    mean='mean',
    std='std',
    skew=skew,
    kurtosis=kurtosis,
    prop_low_0_2=prop_low,
    prop_high_0_8=prop_high
).reset_index()

print(summary_table)


   predicted_anomtype  count  min  max    median      mean       std  \
0                   0    372  0.0  1.0  0.519228  0.517127  0.278714   
1                   1    337  0.0  1.0  0.394707  0.406708  0.241641   
2                   2    191  0.0  1.0  0.682744  0.635228  0.261252   

       skew  kurtosis  prop_low_0_2  prop_high_0_8  
0 -0.001583 -1.099137      0.145161       0.217742  
1  0.351972 -0.537550      0.228487       0.068249  
2 -0.578649 -0.520357      0.073298       0.303665  
